<a href="https://colab.research.google.com/github/SRI-CSL/SusmitJha-UMD23-SummerSchool/blob/main/Packing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!python -m pip install --upgrade --user z3-solver






Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:

from z3 import *
import csv 
import plotly.graph_objects as go
import matplotlib, random
import copy

############################################################
# CONSTANTS
############################################################

# determines how many decimal digits we read for comp size and coordinates
SCALE = 100000 # read 5 digits

# define tolerances for mismatch betweeninput.csv center of gravity and buoyancy
# because it gets multiplied with SCALE * SCALE before being cast as integer, it can support high precision
# problem becomes harder as tolerance becomes lower and runtime increases
TOLx = 0.000001 
TOLy = 0.000001
ATLEASTz = 0.7

# DENSITY OF WATER
DENSITY_WATER = 1000


############################################################
# plotting methods 
############################################################

hex_colors_dic = {}
rgb_colors_dic = {}
hex_colors_only = []
for name, hex in matplotlib.colors.cnames.items():
    #print(name, hex)
    if name == "red" or name ==  "ghostwhite":
      continue # skip red because that we will use for showing overlap, skip ghostwhite for fairing
    hex_colors_only.append(hex)
    hex_colors_dic[name] = hex
    rgb_colors_dic[name] = matplotlib.colors.to_rgb(hex)

def drawBox(vertices, colorID):

  mesh = go.Mesh3d(
        # 8 vertices of a cube
        x=[vertex[0]/SCALE for vertex in vertices],
        y=[vertex[1]/SCALE for vertex in vertices],
        z=[vertex[2]/SCALE for vertex in vertices],

        i = [7, 0, 0, 0, 4, 4, 6, 1, 4, 0, 3, 6],
        j = [3, 4, 1, 2, 5, 6, 5, 2, 0, 1, 6, 3],
        k = [0, 7, 2, 3, 6, 7, 1, 6, 5, 5, 7, 2],
        opacity=0.6,
        color=hex_colors_only[colorID],
        flatshading = True
    )  
  #print(vertices)
  return mesh                 

def drawBoxes(boxes):
  fig = go.Figure(data=[
     drawBox(box,colorID) for box,colorID in zip(boxes,range(len(boxes)))                 
    ])
  fig.show()

def drawComponents(compInfoDictByName):
  boxes = []
  for comp in compInfoDictByName.keys():
    if comp == "fairing":
      print("Warning: The component dictionary should not include a fairing")
    compInfo = compInfoDictByName[comp]
    vertices = getVertices(compInfo)
    boxes.append(vertices)
  drawBoxes(boxes)

############################################################
# box manipulation methods 
############################################################

def getVertices(compInfo):
  vertices = [ 
              [compInfo["tx"] + offset[0], \
               compInfo["ty"] + offset[1], \
               compInfo["tz"] + offset[2] ] for offset in compInfo["VertexOffsets"]]
  return vertices


def getOverlaps(compInfoDictByName):
  overlap = 0
  for i, comp1 in enumerate(compInfoDictByName.keys()):
    compInfo1 = compInfoDictByName[comp1]
    xmin1 = compInfo1["tx"] + compInfo1["minx"]
    xmax1 = compInfo1["tx"] + compInfo1["maxx"]
    ymin1 = compInfo1["ty"] + compInfo1["miny"]
    ymax1 = compInfo1["ty"] + compInfo1["maxy"]
    zmin1 = compInfo1["tz"] + compInfo1["minz"]
    zmax1 = compInfo1["tz"] + compInfo1["maxz"]
    for j, comp2 in enumerate(compInfoDictByName.keys()):
      if j <= i:
        continue
      compInfo2 = compInfoDictByName[comp2]
      xmin2 = compInfo2["tx"] + compInfo2["minx"]
      xmax2 = compInfo2["tx"] + compInfo2["maxx"]
      ymin2 = compInfo2["ty"] + compInfo2["miny"]
      ymax2 = compInfo2["ty"] + compInfo2["maxy"]
      zmin2 = compInfo2["tz"] + compInfo2["minz"]
      zmax2 = compInfo2["tz"] + compInfo2["maxz"]

      if xmax1 >= xmin2 and xmax2 >= xmin1:
        if ymax1 >= ymin2 and ymax2 >= ymin1:
          if zmax1 >= zmin2 and zmax2 >= zmin1:
            overlap = overlap + 1
            print("Overlap between {} and {}: ".format(comp1, comp2))
  print("Total number of overlaps: {}".format(overlap))

def getOldWrongOverlaps(compInfoDictByName):
  overlap = 0
  for comp1 in compInfoDictByName.keys():
    for comp2 in compInfoDictByName.keys():
      if comp1 == comp2:
        continue
      compInfo2 = compInfoDictByName[comp2]
      vertices2 = getVertices(compInfo2)
      xmin2 = min([vertex2[0] for vertex2 in vertices2])
      xmax2 = max([vertex2[0] for vertex2 in vertices2])
      ymin2 = min([vertex2[1] for vertex2 in vertices2])
      ymax2 = max([vertex2[1] for vertex2 in vertices2])
      zmin2 = min([vertex2[2] for vertex2 in vertices2])
      zmax2 = max([vertex2[2] for vertex2 in vertices2])
      compInfo1 = compInfoDictByName[comp1]
      vertices1 = getVertices(compInfo1)
      for vertex1 in vertices1:
        x, y, z = vertex1[0], vertex1[1], vertex1[2]
        if (xmin2 < x < xmax2 and ymin2 < y < ymax2 and zmin2 < z < zmax2):
          overlap = overlap + 1
          print("WRONG Overlap between {} and {}: ".format(comp1, comp2), x, y, z, getVertices(compInfo2))
  print("WRONG Total number of overlaps: {}".format(overlap))


def getCGandCB(compInfoDictByName):
  weightedMass = {}
  weightedBuoyancy = {} 
  totalMass = sum([compInfo["mass_kg"]/SCALE for compInfo in compInfoDictByName.values()]) 
  totalBuoyancy = sum([compInfo["buoyancy_kg"]/SCALE for compInfo in compInfoDictByName.values()]) 
  for t in ["tx", "ty", 'tz']:
    weightedMass[t] = 0
    weightedBuoyancy[t] = 0
    for comp in compInfoDictByName.keys():
      compInfo = compInfoDictByName[comp]
      weightedMass[t] = weightedMass[t] + compInfo[t]/SCALE * compInfo["mass_kg"]/SCALE 
      weightedBuoyancy[t] = weightedBuoyancy[t] + compInfo[t]/SCALE * compInfo["buoyancy_kg"]/SCALE 

  cg = (weightedMass["tx"] / totalMass, weightedMass["ty"] / totalMass, weightedMass["tz"] / totalMass)
  cb = (weightedBuoyancy["tx"] / totalBuoyancy, weightedBuoyancy["ty"] / totalBuoyancy, weightedBuoyancy["tz"] / totalBuoyancy)

  return {"cg":cg,"cb": cb, "mass": totalMass, "buoyancy": totalBuoyancy}

############################################################
# smt formula methods 
############################################################

def defineVars(compInfoDictByName):
  vars = {}
  for comp in compInfoDictByName.keys():
    vars[comp] = {}
    for t in ["tx", "ty", "tz"]:
      vars[comp][t] = Int(comp + '_' + t) 
  return vars

def fairingBoundConstraints(compInfoDictByName, fairing, smtVars):
  constraints = []
  
  fairingVertices = getVertices(fairing)
  xmin = min([fairingVertex[0] for fairingVertex in fairingVertices])
  xmax = max([fairingVertex[0] for fairingVertex in fairingVertices])
  ymin = min([fairingVertex[1] for fairingVertex in fairingVertices])
  ymax = max([fairingVertex[1] for fairingVertex in fairingVertices])
  zmin = min([fairingVertex[2] for fairingVertex in fairingVertices])
  zmax = max([fairingVertex[2] for fairingVertex in fairingVertices])
  print("Fairing:", xmin, xmax, ymin, ymax, zmin, zmax )
  for comp in compInfoDictByName.keys():
    compInfo = compInfoDictByName[comp]
    ldf = compInfo["VertexOffsets"][0]
    rub = compInfo["VertexOffsets"][6]

    constraints.append(smtVars[comp]["tx"] >= xmin - ldf[0])
    constraints.append(smtVars[comp]["tx"] <= xmax - rub[0])
    constraints.append(smtVars[comp]["ty"] >= ymin - ldf[1])
    constraints.append(smtVars[comp]["ty"] <= ymax - rub[1])
    constraints.append(smtVars[comp]["tz"] >= zmin - ldf[2])
    constraints.append(smtVars[comp]["tz"] <= zmax - rub[2])

  return constraints

def nonOverlapConstraints(compInfoDictByName, smtVars):
  constraints = []  
  for i, comp1 in enumerate(compInfoDictByName.keys()):
    compInfo1 = compInfoDictByName[comp1]
    xmin1 = smtVars[comp1]["tx"] + compInfo1["minx"]
    xmax1 = smtVars[comp1]["tx"] + compInfo1["maxx"]
    ymin1 = smtVars[comp1]["ty"] + compInfo1["miny"]
    ymax1 = smtVars[comp1]["ty"] + compInfo1["maxy"]
    zmin1 = smtVars[comp1]["tz"] + compInfo1["minz"]
    zmax1 = smtVars[comp1]["tz"] + compInfo1["maxz"]
    for j, comp2 in enumerate(compInfoDictByName.keys()):
        if j <= i:
          continue
        compInfo2 = compInfoDictByName[comp2]
        xmin2 = smtVars[comp2]["tx"] + compInfo2["minx"]
        xmax2 = smtVars[comp2]["tx"] + compInfo2["maxx"]
        ymin2 = smtVars[comp2]["ty"] + compInfo2["miny"]
        ymax2 = smtVars[comp2]["ty"] + compInfo2["maxy"]
        zmin2 = smtVars[comp2]["tz"] + compInfo2["minz"]
        zmax2 = smtVars[comp2]["tz"] + compInfo2["maxz"]

        constraints.append(Or(xmax1 < xmin2, xmax2 < xmin1, \
                              ymax1 < ymin2, ymax2 < ymin1, \
                              zmax1 < zmin2, zmax2 < zmin1))
  return constraints


def matchCGandCBconstraints(compInfoDictByName, smtVars):
  constraints = []
  weightedMass = {}
  weightedBuoyancy = {} 
  totalMass = sum([compInfo["mass_kg"] for compInfo in compInfoDictByName.values()]) 
  totalBuoyancy = sum([compInfo["buoyancy_kg"] for compInfo in compInfoDictByName.values()]) 
  for t in ["tx", "ty", 'tz']:
    weightedMass[t] = IntVal(0)
    weightedBuoyancy[t] = IntVal(0)
    for comp in compInfoDictByName.keys():
      compInfo = compInfoDictByName[comp]
      weightedMass[t] = weightedMass[t] + smtVars[comp][t]* IntVal(compInfo["mass_kg"])
      weightedBuoyancy[t] = weightedBuoyancy[t] + smtVars[comp][t]* IntVal(compInfo["buoyancy_kg"])

  #cg = (weightedMass["tx"] / totalMass, weightedMass["ty"] / totalMass, weightedMass["tz"] / totalMass)
  #cb = (weightedBuoyancy["tx"] / totalBuoyancy, weightedBuoyancy["ty"] / totalBuoyancy, weightedBuoyancy["tz"] / totalBuoyancy)

  # assume total buoyancy and totalmass are close to each other 
  # that is totalMass ~= totalBuoyancy

  deltaX = IntVal(TOLx * SCALE * SCALE) # one SCALE for the location and one for the mass/buoyancy
  deltaY = IntVal(TOLy * SCALE * SCALE)
  deltaZ = IntVal(ATLEASTz * SCALE * SCALE)

  lhs = weightedMass["tx"] 
  rhs = weightedBuoyancy["tx"] 
  constraints.append(And(lhs  <= rhs + deltaX,lhs >= rhs - deltaX))

  lhs = weightedMass["ty"] 
  rhs = weightedBuoyancy["ty"] 
  constraints.append(And(lhs  <= rhs + deltaY,lhs >= rhs - deltaY))

  lhs = weightedMass["tz"] 
  rhs = weightedBuoyancy["tz"]
  constraints.append(lhs  <= rhs  - deltaZ) # make it a bit more lower because mass and buoy may be not same

  constraints.append(Int('mass_x') == weightedMass["tx"])
  constraints.append(Int('mass_y') == weightedMass["ty"])
  constraints.append(Int('mass_z') == weightedMass["tz"])
  constraints.append(Int('buoy_x') == weightedBuoyancy["tx"])
  constraints.append(Int('buoy_y') == weightedBuoyancy["ty"])
  constraints.append(Int('buoy_z') == weightedBuoyancy["tz"])


  return constraints

############################################################
# smt solving methods 
############################################################

def packUsingSMT(compInfoDictByName, fairing):
  newCompInfoDictByName = copy.deepcopy(compInfoDictByName)
  smt = Solver()
  smtVars = defineVars(compInfoDictByName)
  smt.add(fairingBoundConstraints(compInfoDictByName, fairing, smtVars)) 
  smt.add(nonOverlapConstraints(compInfoDictByName,smtVars))
  smt.add(matchCGandCBconstraints(compInfoDictByName,smtVars))
  if smt.check() == sat:
    model = smt.model()
    print(model)
    for comp in compInfoDictByName.keys():
      for t in ["tx", "ty", "tz"]:
        print(smtVars[comp][t], model[smtVars[comp][t]]) 
        newCompInfoDictByName[comp][t] = int(model[smtVars[comp][t]].as_string())
    return newCompInfoDictByName
  else:
    print(smt.assertions())
    print("No Solution")
    return None



############################################################
# I/O methods 
############################################################

def readInputComponentDetails(filename):
  compInfoDictByName = {}
  with open(filename, mode ='r') as inp:
    componentsInfo = csv.DictReader(inp)
    for compInfo in componentsInfo:
      # each corner's coordinate is specified as x,y,z
      #up-down left-right back-front 
      attrList = ['tx', 'ty', 'tz', 'minx', 'maxx', 'miny', 'maxy', 'minz', 'maxz', "mass_kg" ]
      for attrb in attrList:
        compInfo[attrb] = int(float(compInfo[attrb]) * SCALE) # only look at 5 decimal digits
      tx, ty, tz, minx, maxx, miny, maxy, minz, maxz, mass  = [compInfo[q] for q in attrList]
      # convert volume to boyancy in Kg and then scale 
      compInfo["buoyancy_kg"]  = int(float(compInfo["volume_m3"]) * DENSITY_WATER * SCALE)

      compInfo["VertexOffsets"] = [] #{}
      
      # compInfo["VertexOffsets"]['lub']   = [minx,  maxy,  maxz]
      # compInfo["VertexOffsets"]['luf']   = [minx,  maxy,  minz]
      # compInfo["VertexOffsets"]['rub']   = [maxx,  maxy,  maxz]
      # compInfo["VertexOffsets"]['ruf']   = [maxx,  maxy,  minz]
      # compInfo["VertexOffsets"]['ldb']   = [minx,  miny,  maxz]
      # compInfo["VertexOffsets"]['ldf']   = [minx,  miny,  minz]
      # compInfo["VertexOffsets"]['rdb']   = [maxx,  miny,  maxz]
      # compInfo["VertexOffsets"]['rdf']   = [maxx,  miny,  minz]
      
      # this was redone to make plotly happy; 
      # plotly trinagulation depends on the ordering of the vertices
      # because of i,j,k setup
      compInfo["VertexOffsets"].append([minx,  miny,  minz])
      compInfo["VertexOffsets"].append([minx,  maxy,  minz])
      compInfo["VertexOffsets"].append([maxx,  maxy,  minz])
      compInfo["VertexOffsets"].append([maxx,  miny,  minz])
      compInfo["VertexOffsets"].append([minx,  miny,  maxz])
      compInfo["VertexOffsets"].append([minx,  maxy,  maxz])
      compInfo["VertexOffsets"].append([maxx,  maxy,  maxz])
      compInfo["VertexOffsets"].append([maxx,  miny,  maxz])
      if compInfo['name'] == "fairing":
        fairing = compInfo
      else:
        compInfoDictByName[compInfo['name']] = compInfo
  return compInfoDictByName, fairing


In [ ]:

############################################################
# run script
############################################################

compInfoDictByName, fairing = readInputComponentDetails('input.csv')
#compInfoDictByName, fairing = readInputComponentDetails('input-simple.csv')

drawComponents(compInfoDictByName)
getOverlaps(compInfoDictByName)
print("CG-CB-Mass-Buoyancy are {}".format(getCGandCB(compInfoDictByName)))

newCompInfoDictByName = packUsingSMT(compInfoDictByName, fairing)
if newCompInfoDictByName is not None:
  drawComponents(newCompInfoDictByName)
  getOverlaps(newCompInfoDictByName)
  print("CG-CB-Mass-Buoyancy are {}".format(getCGandCB(newCompInfoDictByName)))
  print(newCompInfoDictByName.keys())

Overlap between Fwd Elec Assembly and FloatCylOBS2: 
Overlap between PitchMassTube1 and PitchMass1: 
Overlap between PitchMassTube2 and PitchMass2: 
Total number of overlaps: 3
CG-CB-Mass-Buoyancy are {'cg': (4.027404417814834, 0.0, -0.08678906113860096), 'cb': (4.027246969238212, -2.7781273293425336e-19, -0.029072573797429294), 'mass': 2430.2894999999994, 'buoyancy': 2397.7799999999997}
Fairing: -100000 919999 -55000 55000 -59000 51000
[Thruster_tz = -26335,
 FloatCyl4_ty = 1048,
 FloatCylOBS2_tz = -1417,
 FloatCylOBS2_tx = 26946,
 Thruster_ty = 34983,
 Rudder_tx = 565189,
 OBS_tz = -1422,
 PitchMass1_tx = 336145,
 INS_DVL_ty = 36221,
 BatteryPack2_tx = 112447,
 BatteryPack1_ty = -5861,
 FloatBrick2_ty = -32500,
 BatteryPack2_ty = -29370,
 FloatBrick1_tx = -70000,
 PitchMassTube1_tx = 284948,
 PitchMass2_tx = 427199,
 FloatCyl5_tz = -1417,
 FloatBrick2_tz = -27019,
 BatteryPack1_tz = -26472,
 BatteryPack2_tz = -26422,
 Aft Elec Assembly_tx = 919989,
 FloatCyl1_ty = 1241,
 FloatCyl2_tx

Total number of overlaps: 0
CG-CB-Mass-Buoyancy are {'cg': (3.486901691171361, -0.05753222523900961, -0.16019800532405715), 'cb': (3.5341776842554364, -0.058312256396333285, -0.10628328644079106), 'mass': 2430.2894999999994, 'buoyancy': 2397.7799999999997}
dict_keys(['CommunicationTransponder', 'BatteryPack1', 'Fwd Elec Assembly', 'FloatBrick1', 'Aft Elec Assembly', 'BatteryPack2', 'FloatBrick2', 'OBS', 'FloatCylOBS1', 'FloatCylOBS2', 'PitchMassTube1', 'PitchMass1', 'PitchMassTube2', 'PitchMass2', 'INS_DVL', 'Thruster', 'FloatCyl5', 'FloatCyl1', 'FloatCyl2', 'FloatCyl3', 'FloatCyl4', 'Rudder'])
